In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.6 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
     |████████████████████████████████| 61 kB 443 kB/s 
     |████████████████████████████████| 895 kB 54.6 MB/s 
     |████████████████████████████████| 3.3 MB 42.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/df_5000.csv')

In [ ]:
df = dataframe[:30]

In [ ]:
split_arr = []
for i in range(30):
  text = df['QUESTION'][i]
  split_arr.append(text.split("[BLANK]"))

df_split = pd.DataFrame(split_arr,columns=['split1','split2'])

In [ ]:
df = pd.concat([df,df_split],axis=1)
df = df[['QUESTION','ANSWER','split1','split2']]

In [ ]:
df

,QUESTION,ANSWER,split1,split2
0,Increased content consumption on mobile device...,viewing,Increased content consumption on mobile device...,reached 33% of total online video plays in th...
1,National Australia Bank's earnings prospects w...,offsetting,National Australia Bank's earnings prospects w...,a weakness in consumer banking. Market income...
2,The success of TransCanada's $15 billion claim...,overturns,The success of TransCanada's $15 billion claim...,the denial. Most NAFTA disputes settle for an...
3,German carmakers may continue to lead factory ...,ran,German carmakers may continue to lead factory ...,at full capacity last year as production fell...
4,New Cotai resorts have helped fuel citywide re...,attract,New Cotai resorts have helped fuel citywide re...,guests and revenue volatility as recent cityw...
5,The combination of higher interest rates and p...,purchasing,The combination of higher interest rates and p...,Chubb Corp. and boosted the acquired portfoli...
6,Saudi Arabia baby food sales will surge 11% a ...,speed,Saudi Arabia baby food sales will surge 11% a ...,up implementation of policies aimed at boosti...
7,Nortel Networks Inc. is likely to request by J...,denied,Nortel Networks Inc. is likely to request by J...,such a request. The 3rd Circuit now must deci...
8,Makers of brand-name drugs and biologics will ...,delayed,Makers of brand-name drugs and biologics will ...,"multiple Obamacare-imposed taxes, including t..."
9,Two California owners of the shuttered San Ono...,supplied,Two California owners of the shuttered San Ono...,"defective steam generators, was held liable b..."


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',return_dict = True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
labels = []
for i in range(30):
  labels_one_sentence = []
  text = df['split1'][i] + tokenizer.mask_token + df['split2'][i]

  input = tokenizer.encode_plus(text, return_tensors = "pt")
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)


  output = model(**input)
  logits = output.logits

  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]

  top_5 = torch.topk(mask_word, 5, dim = 1)[1][0]
  for token in top_5:
    word = tokenizer.decode([token])
    labels_one_sentence.append(word)

  labels.append(labels_one_sentence)

In [ ]:
df_label = pd.DataFrame(labels)


In [ ]:
df_answer = pd.concat([df['ANSWER'],df_label],axis=1)
df_answer

,ANSWER,0,1,2,3,4
0,viewing,devices,content,videos,games,users
1,offsetting,despite,but,and,indicating,showing
2,overturns,accepts,wins,rejects,disputes,challenges
3,ran,was,were,worked,remained,stood
4,attract,its,attract,the,accommodate,cater
5,purchasing,acquiring,purchasing,buying,acquisition,creating
6,speed,speed,drive,keep,hold,speeds
7,denied,denied,rejected,refused,made,granted
8,delayed,removed,eliminated,increased,reduced,repealed
9,supplied,manufactured,produced,manufactures,built,installed


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Tdf_5000.csv')

In [ ]:
split_arr = []
for i in range(30):
  text = df_test['QUESTION'][i]
  split_arr.append(text.split("[BLANK]"))

split_arr
df_T = df_test.iloc[:30,:]
df_split = pd.DataFrame(split_arr,columns=['split1','split2'])
df_T = pd.concat([df_T,df_split],axis=1)
df_T = df_T[['QUESTION','ANSWER','split1','split2']]
df_T

,QUESTION,ANSWER,split1,split2
0,While construction has begun on the Nord Strea...,grant,While construction has begun on the Nord Strea...,the necessary approvals. Nord Stream 2 is als...
1,"A case against 47 people, including OMV execut...",passed,"A case against 47 people, including OMV execut...","on to the buyer. Yet, the allegations could a..."
2,Property insurers are positioning themselves t...,tied,Property insurers are positioning themselves t...,to an unsustainable system. Modest reforms pr...
3,Low-cost European airlines averaged 9% capacit...,garner,Low-cost European airlines averaged 9% capacit...,"higher yields. EasyJet added only 2.5%, compa..."
4,Some of the junket operators integral to Macau...,plunged,Some of the junket operators integral to Macau...,42% in the first quarter. With close relation...
5,Recovering state budgets are supporting road-r...,led,Recovering state budgets are supporting road-r...,"by pavement spending, up 8.5%. Pavement const..."
6,An increase in the number of U.S. hotel rooms ...,experience,An increase in the number of U.S. hotel rooms ...,"this decline first, since expansion there has..."
7,"BlackRock, Prudential Financial and other mort...",narrowing,"BlackRock, Prudential Financial and other mort...",the $98 billion lawsuit against the bank in i...
8,Legislation that would give the U.S. Departmen...,controls,Legislation that would give the U.S. Departmen...,"either chamber of Congress, this measure favo..."
9,Neither side of the legal challenge to the EPA...,committed,Neither side of the legal challenge to the EPA...,to seeing the process through to the end. Law...


In [ ]:
labels = []
for i in range(30):
  labels_one_sentence = []
  text = df_T['split1'][i] + tokenizer.mask_token + df_T['split2'][i]

  input = tokenizer.encode_plus(text, return_tensors = "pt")
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)


  output = model(**input)
  logits = output.logits

  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]

  top_5 = torch.topk(mask_word, 5, dim = 1)[1][0]
  for token in top_5:
    word = tokenizer.decode([token])
    labels_one_sentence.append(word)

  labels.append(labels_one_sentence)

In [ ]:
df_label = pd.DataFrame(labels)


In [ ]:
df_answer = pd.concat([df_T['ANSWER'],df_label],axis=1)
df_answer

,ANSWER,0,1,2,3,4
0,grant,obtain,issue,get,receive,give
1,passed,passed,passing,pass,sold,passes
2,tied,exposed,vulnerable,connected,committed,susceptible
3,garner,achieve,provide,deliver,offer,produce
4,plunged,rose,increased,grew,fell,dropped
5,led,followed,accompanied,backed,aided,supported
6,experience,see,experience,notice,face,have
7,narrowing,in,ending,dismissing,regarding,ruling
8,controls,controls,in,wins,dominates,leads
9,committed,committed,dedicated,committing,devoted,close


In [ ]:
str = dataframe['QUESTION'][0].replace('[BLANK]',dataframe['ANSWER'][0])
splitter = str.split(' ')

In [ ]:
text = str.split(splitter[0])

In [ ]:
len(text)

2

In [ ]:
len(splitter)

73

In [ ]:
labels = []

for j in range(2):
  str = dataframe['QUESTION'][j].replace('[BLANK]',dataframe['ANSWER'][j])
  splitter = str.split(' ')
  for i in range(len(splitter)):
    labels_one_sentence = []
    s = splitter[i]
    t = str.split(s)
    print("splitter ",i," ",splitter[i])
    text = t[0] + tokenizer.mask_token + t[1]
    print("t0 ",t[0],"\n t1 ", t[1],"\n\n")

    input = tokenizer.encode_plus(text, return_tensors = "pt")
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)


    output = model(**input)
    logits = output.logits

    softmax = F.softmax(logits, dim = -1)
    mask_word = softmax[0, mask_index, :]

    top_5 = torch.topk(mask_word, 5, dim = 1)[1][0]
    for token in top_5:
      word = tokenizer.decode([token])
      labels_one_sentence.append(word)

    labels.append(labels_one_sentence)

splitter  0   Increased
t0   
 t1   content consumption on mobile devices may further drive distributors such as cable and telecom operators to release over-the-top mobile video offerings, while shifting ad budgets to the Internet. Smartphone and tablet viewing reached 33% of total online video plays in the fourth quarter, according to Ooyala, doubling from 17% in 4Q13. A number of content distributors have responded to this trend, with Verizon announcing a mobile video service to debut this summer. 


splitter  1   content
t0  Increased  
 t1   consumption on mobile devices may further drive distributors such as cable and telecom operators to release over-the-top mobile video offerings, while shifting ad budgets to the Internet. Smartphone and tablet viewing reached 33% of total online video plays in the fourth quarter, according to Ooyala, doubling from 17% in 4Q13. A number of  


splitter  2   consumption
t0  Increased content  
 t1   on mobile devices may further drive distributor

In [ ]:
df_labels = pd.DataFrame(labels)
df_labels

,0,1,2,3,4
0,increased,increasing,rising,low,high
1,ad,media,video,advertising,content
2,availability,content,demand,streaming,distribution
3,increased,-,.,reduced,new
4,mobile,these,such,wireless,digital
...,...,...,...,...,...
146,$,new,proposed,section,initial
147,government,federal,annual,new,capital
148,cap,return,limit,tax,loss
149,and,-,says,:,said


In [ ]:
s1 = dataframe['QUESTION'][0].replace('[BLANK]',dataframe['ANSWER'][0]).split(' ')
s2 = dataframe['QUESTION'][1].replace('[BLANK]',dataframe['ANSWER'][1]).split(' ')
s3 = s1 +s2
df_splitter = pd.DataFrame(s3,columns=['ANSWER'])
df_splitter

,ANSWER
0,Increased
1,content
2,consumption
3,on
4,mobile
...,...
146,proposed
147,6-bp
148,levy
149,on


In [ ]:
df_answers = pd.concat([df_splitter,df_labels],axis=1)
df_answers


,ANSWER,0,1,2,3,4
0,Increased,increased,increasing,rising,low,high
1,content,ad,media,video,advertising,content
2,consumption,availability,content,demand,streaming,distribution
3,on,increased,-,.,reduced,new
4,mobile,mobile,these,such,wireless,digital
...,...,...,...,...,...,...
146,proposed,$,new,proposed,section,initial
147,6-bp,government,federal,annual,new,capital
148,levy,cap,return,limit,tax,loss
149,on,and,-,says,:,said


In [ ]:
df_answers.to_csv('Answers_BertPrediction.csv')